In [2]:
###### Calculates LOS and outcome proportions for ward and ICU compartments

# Replace FOLDER to read SRAG csv from datasus - SRAG dataset can be found at https://opendatasus.saude.gov.br/dataset/bd-srag-2020
# Data dictionary for codes from SRAG dataset can be found at the same site

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow

pd.set_option('display.max_columns', 156)

df_srag_raw = pd.read_csv(r'FOLDER\INFLUD-31-08-2020.csv', sep=';', low_memory=False) #source https://opendatasus.saude.gov.br/dataset/bd-srag-2020#

#FORMATA DATAS
df_srag_raw['DT_SIN_PRI'] = pd.to_datetime(df_srag_raw['DT_SIN_PRI'], format='%d/%m/%Y')
df_srag_raw['DT_NOTIFIC'] = pd.to_datetime(df_srag_raw['DT_NOTIFIC'], format='%d/%m/%Y')
df_srag_raw['DT_SAIDUTI'] = pd.to_datetime(df_srag_raw['DT_SAIDUTI'], format='%d/%m/%Y')
df_srag_raw['DT_INTERNA'] = pd.to_datetime(df_srag_raw['DT_INTERNA'], format='%d/%m/%Y', errors='coerce')
df_srag_raw['DT_ENTUTI'] = pd.to_datetime(df_srag_raw['DT_ENTUTI'], format='%d/%m/%Y', errors='coerce')
df_srag_raw['DT_EVOLUCA'] = pd.to_datetime(df_srag_raw['DT_EVOLUCA'], format='%d/%m/%Y', errors='coerce')

In [3]:
#SELECTS ONLY COVID CONFIRMED CASES
df_srag_covid = df_srag_raw.loc[df_srag_raw.CLASSI_FIN == 5, :]

In [4]:
###### MAPS CODES OF ICU AND OUTCOMES, DISCRETIZE AGES IN TWO GROUPS (ELDERLY vs YOUNG)
######Classifica os codigos de UTI, evolucao e idade
# Pergunta campo 47 - Internado em UTI?

#1-Sim
#2-Não
#9-Ignorado

#Pergunta campo 74 - Evolução do caso

#1-Cura 
#2-Óbito 
#3-Óbito por outras causas 
#9-Ignorado

def classifica_idade(entrada):
    if entrada >= 60:
        return 'idoso'
    elif entrada < 60:
        return 'jovem'
    
def classifica_evolucao(entrada):
    if entrada == 1:
        return 'Cura'
    elif entrada == 2:
        return 'Óbito'
    elif entrada == 3:
        return 'Óbito por outras causas'
    elif entrada == 9:
        return 'ignorado'

def classifica_UTI(entrada):
    if entrada == 1:
        return 'Sim_UTI'
    elif entrada == 2:
        return 'Nao_uti'
    elif entrada == 9:
        return 'ignorado'

df_srag_covid['classifica_idade'] = df_srag_covid['NU_IDADE_N'].apply(classifica_idade)
df_srag_covid['classifica_evolucao'] = df_srag_covid['EVOLUCAO'].apply(classifica_evolucao)
df_srag_covid['classifica_UTI'] = df_srag_covid['UTI'].apply(classifica_UTI)

C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [5]:
#Hierarquiza os casos em tipos ['classifica_idade', 'classifica_UTI', 'classifica_evolucao']

idade_uti_evolucao = df_srag_covid.loc[(df_srag_covid.classifica_UTI != 'ignorado') & (df_srag_covid.classifica_evolucao != 'ignorado')].groupby(['classifica_idade', 'classifica_UTI', 'classifica_evolucao']).DT_NOTIFIC.count()
idade_uti_evolucao

classifica_idade  classifica_UTI  classifica_evolucao    
idoso             Nao_uti         Cura                       43149
                                  Óbito                      27981
                                  Óbito por outras causas       54
                  Sim_UTI         Cura                       14226
                                  Óbito                      40673
                                  Óbito por outras causas       65
jovem             Nao_uti         Cura                       69300
                                  Óbito                       8369
                                  Óbito por outras causas       32
                  Sim_UTI         Cura                       19718
                                  Óbito                      15063
                                  Óbito por outras causas       51
Name: DT_NOTIFIC, dtype: int64

In [6]:
############ CALCULATES TIMEDELTAS FOR LOS CALCULATIONS

In [7]:
df_srag_covid['admission_to_outcome'] = (df_srag_covid['DT_EVOLUCA'] - df_srag_covid['DT_INTERNA']).astype('timedelta64[D]')
df_srag_covid['ICU_admission_to_ICU_dischage'] = (df_srag_covid['DT_SAIDUTI'] - df_srag_covid['DT_ENTUTI']).astype('timedelta64[D]')
df_srag_covid['admission_to_ICU_admission'] = (df_srag_covid['DT_ENTUTI'] - df_srag_covid['DT_INTERNA']).astype('timedelta64[D]')
df_srag_covid['ICU_dischage_to_outcome'] = (df_srag_covid['DT_EVOLUCA'] - df_srag_covid['DT_SAIDUTI']).astype('timedelta64[D]')

C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [8]:

df_srag_covid.groupby(['classifica_idade','classifica_UTI','classifica_evolucao'])['admission_to_outcome', 'admission_to_ICU_admission','ICU_admission_to_ICU_dischage', 'ICU_dischage_to_outcome'].mean().loc[pd.IndexSlice[:,['Nao_uti', 'Sim_UTI'],['Cura', 'Óbito']],:]

C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


admission_to_outcome  \
classifica_idade classifica_UTI classifica_evolucao                         
idoso            Nao_uti        Cura                             9.394910   
                                Óbito                            9.201503   
                 Sim_UTI        Cura                            16.873551   
                                Óbito                           12.545638   
jovem            Nao_uti        Cura                             7.494861   
                                Óbito                            9.475104   
                 Sim_UTI        Cura                            13.758354   
                                Óbito                           13.109241   

                                                     admission_to_ICU_admission  \
classifica_idade classifica_UTI classifica_evolucao                               
idoso            Nao_uti        Cura                                        NaN   
                                Óbito                                       NaN   
                 Sim_UTI        Cura                                   0.987291   
                                Óbito                                  1.477277   
jovem            Nao_uti        Cura                                        NaN   
                                Óbito                                       NaN   
                 Sim_UTI        Cura                                   0.753683   
                                Óbito                                  1.421647   

                                                     ICU_admission_to_ICU_dischage  \
classifica_idade classifica_UTI classifica_evolucao                                  
idoso            Nao_uti        Cura                                           NaN   
                                Óbito                                          NaN   
                 Sim_UTI        Cura                                      9.847628   
                                Óbito                                     9.868753   
jovem            Nao_uti        Cura                                           NaN   
                                Óbito                                          NaN   
                 Sim_UTI        Cura                                      8.736200   
                                Óbito                                    10.720939   

                                                     ICU_dischage_to_outcome  
classifica_idade classifica_UTI classifica_evolucao                           
idoso            Nao_uti        Cura                                     NaN  
                                Óbito                                    NaN  
                 Sim_UTI        Cura                                6.572384  
                                Óbito                               0.662800  
jovem            Nao_uti        Cura                                     NaN  
                                Óbito                                    NaN  
                 Sim_UTI        Cura                                4.594393  
                                Óbito                               0.473185

In [ ]:
los_WARD_survive_i = 9.394910
los_WARD_survive_j = 7.494861
los_WARD_death_i = 9.201503
los_WARD_death_j = 9.475104
los_ICU_survive_i = 9.847628
los_ICU_survive_j = 8.736200
los_ICU_death_i = 9.868753
los_ICU_death_j = 10.720939
los_discharged_ICU_survive_i = 6.572384
los_discharged_ICU_survive_j = 4.594393

In [9]:
############## COUNT OUTCOMES
outcome_multindex_count = df_srag_covid.loc[(df_srag_covid.classifica_UTI != 'ignorado') & (df_srag_covid.classifica_evolucao != 'ignorado') & (df_srag_covid.classifica_evolucao != 'Óbito por outras causas')].groupby(['classifica_idade', 'classifica_UTI', 'classifica_evolucao'])['DT_NOTIFIC','admission_to_outcome'] .count()

C:\Users\Vito\Downloads\WPy64-3760\python-3.7.6.amd64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [21]:
############ CALCULATES PROPORTIONS OF EACH OUTCOME FOR COMPARTMENTS

outcome_multindex_count.groupby(level=(0,1)).apply(lambda x:  x / x.sum())#.apply(lambda x:  100*x / x.sum())

DT_NOTIFIC  \
classifica_idade classifica_UTI classifica_evolucao               
idoso            Nao_uti        Cura                   0.606622   
                                Óbito                  0.393378   
                 Sim_UTI        Cura                   0.259130   
                                Óbito                  0.740870   
jovem            Nao_uti        Cura                   0.892248   
                                Óbito                  0.107752   
                 Sim_UTI        Cura                   0.566919   
                                Óbito                  0.433081   

                                                     admission_to_outcome  
classifica_idade classifica_UTI classifica_evolucao                        
idoso            Nao_uti        Cura                             0.592898  
                                Óbito                            0.407102  
                 Sim_UTI        Cura                             0.253179  
                                Óbito                            0.746821  
jovem            Nao_uti        Cura                             0.886406  
                                Óbito                            0.113594  
                 Sim_UTI        Cura                             0.559564  
                                Óbito                            0.440436

In [ ]:
WARD_survive_proportion_i = 0.606622
WARD_survive_proportion_j = 0.892248
ICU_survive_proportion_i = 0.259130
ICU_survive_proportion_j = 0.566919
